In [156]:
import pandas as pd
import datetime
import itertools
import requests
from bs4 import BeautifulSoup
import re

In [157]:
dir_url = "https://www.karriere.at/jobs?page=2"
hdr = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

# Request page content
dir_page_response = requests.get(dir_url, timeout=120, headers = hdr)

# Parse and turn into soup
dir_page_soup = BeautifulSoup(dir_page_response.content, 'html.parser')

# Returns max. number of pages to scrape
max_anzahl = dir_page_soup.find_all(class_ = 'm-pagination__meta')
max_anzahl = int(max_anzahl[0].contents[0].split()[2]) + 1

In [158]:
job_names = []
company_names = []
location_names = []
description_names = []
job_card_links = []

for i in range(1, 5):
    dir_url = 'https://www.karriere.at/jobs?page=' + str(i)
    dir_page_response = requests.get(dir_url, timeout=120, headers = hdr)
    dir_page_soup = BeautifulSoup(dir_page_response.content, 'html.parser')
    job_name = dir_page_soup.find_all(class_= 'm-jobsListItem__titleLink')
    company_name = dir_page_soup.find_all(class_= 'm-jobsListItem__company')
    location_name = dir_page_soup.find_all(class_= 'm-jobsListItem__wrap')
    description_name = dir_page_soup.find_all(class_= 'm-jobsListItem__snippet')
    job_card_link = dir_page_soup.find_all(class_= 'm-jobsListItem__titleLink')
    company_names.append(company_name)
    location_names.append(location_name)
    description_names.append(description_name)
    job_card_links.append(job_card_link)
    job_names.append(job_name)

In [159]:
merged_company_names = list(itertools.chain(*company_names))
company_names_clean = []
for i in range(len(merged_company_names)):
    company_names_clean.append(merged_company_names[i].contents[1].text)

In [160]:
merged_job_names = list(itertools.chain(*job_names))
job_names_clean = []
for i in range(len(merged_job_names)):
    job_names_clean.append(merged_job_names[i].contents[0].strip())

In [161]:
merged_location_names = list(itertools.chain(*location_names))
location_names_clean = []
for i in range(len(merged_location_names)):
    location_names_clean.append(merged_location_names[i].contents[1].text)

In [162]:
merged_description_names = list(itertools.chain(*description_names))
description_names_clean = []
for i in range(len(merged_description_names)):
    description_names_clean.append(merged_description_names[i].contents[0].strip())

In [163]:
merged_link_names = list(itertools.chain(*job_card_links))
job_card_links_clean = []
for i in range(len(merged_link_names)):
    job_card_links_clean.append(str(merged_link_names[i]).split())
merged_card_links = list(itertools.chain(*job_card_links_clean))
find_all_links = [s for s in merged_card_links if 'href=' in s]
find_all_links = list(map( lambda x: x.replace("href=", ''), find_all_links))
find_all_links = list(map( lambda x: x.replace(">", ''), find_all_links))
find_all_links = list(map( lambda x: x.replace('"', ''), find_all_links))

In [164]:
content_job_page = []

for i in find_all_links:
    
    # Request page content
    dir_page_response = requests.get(i, timeout=120, headers = hdr)

    #Parse and turn into soup
    dir_page_soup = BeautifulSoup(dir_page_response.content, 'html.parser')
    
    # content-page to get salary
    content_page = dir_page_soup.find_all('div', attrs={'class':'m-jobContent__jobText'})
    content_job_page.append(content_page)
        
merged_salary_list = list(itertools.chain(*content_job_page))

In [165]:
df = pd.DataFrame(list(zip(company_names_clean, job_names_clean, location_names_clean,  merged_salary_list)))
df.columns = ['company', 'job', 'location', 'page_content']

In [166]:
df['page_content'] = df['page_content'].astype('str') 
content_list = df['page_content'].to_list()
regex_salary = []
for i in range(len(content_list)):
    if (bool(re.search(r'\d+[,|.]+[0-9]{3}?(-|,|.)?([0-9]{2}|-)', content_list[i]))) == True:
        regex_salary.append(re.search(r'\d+[,|.]+[0-9]{3}?(-|,|.)?([0-9]{2}|-)', content_list[i]).group(0))
            
    elif (bool(re.search(r'(EUR|€)+\s+\d+[0-9]{1,3}?(.|,)?(\d{3})', content_list[i]))) == True:
        regex_salary.append(re.search(r'[EUR|€]+\s+\d+[0-9]{1,3}?(.|,)?(\d{3})', content_list[i]).group(0))
        
    elif (bool(re.search(r'[EUR|€]+\s+\d+[.|,]+[0-9]{1,3}?(.|,)?(\d{2})', content_list[i]))) == True:
        regex_salary.append(re.search(r'[EUR|€]+\s+\d+[.|,|]+[0-9]{1,3}?(.|,)?(\d{2})', content_list[i]).group(0))
        
    elif (bool(re.search(r'[EUR|€]+\s+[0-9]{1,3}?(.|,)?(-)', content_list[i]))) == True:
        regex_salary.append(re.search(r'[EUR|€]+\s+[0-9]{1,3}?(.|,)?(-)', content_list[i]).group(0))
        
    elif (bool(re.search(r'(EUR|€)+\s+\d+[0-9]{1,3}', content_list[i]))) == True:
        regex_salary.append(re.search(r'(EUR|€)+\s+\d+[0-9]{1,3}', content_list[i]).group(0))
            
    elif (bool(re.search(r'[\d{1,2}|\d{1}]+[,|.]+[0-9]{3}?(\s+brutto|\s+Brutto)', content_list[i]))) == True:
        regex_salary.append(re.search(r'[\d{1,2}|\d{1}]+[,|.]+[0-9]{3}?(\s+brutto|\s+Brutto)', content_list[i]).group(0))     
        
    elif (bool(re.search(r'\d+[,|.]+[0-9]{3}?(-|,|.)?(\s+EUR|\s+€)', content_list[i]))) == True:
        regex_salary.append(re.search(r'\d+[,|.]+[0-9]{3}?(-|,|.)?(\s+EUR|\s+€)', content_list[i]).group(0))
           
    elif (bool(re.search(r'\d+[0-9]{3}?[-|,|.]+\d{2}', content_list[i]))) == True:
        regex_salary.append(re.search(r'\d+[0-9]{3}?[-|,|.]+\d{2}', content_list[i]).group(0))
        
    elif (bool(re.search(r'\d+[,|.]+[0-9]{3}?(-|,|.)', content_list[i]))) == True:
        regex_salary.append(re.search(r'\d+[,|.]+[0-9]{3}?(-|,|.)', content_list[i]).group(0))
        
    elif (bool(re.search(r'\d+[0-9]{3}?(-|,|.)?(-)?(-)', content_list[i]))) == True:
        regex_salary.append(re.search(r'\d+[0-9]{3}?(-|,|.)?(-)?(-)', content_list[i]).group(0))
                       
    else:
        regex_salary.append('False')

        
# Exception-handling

## Exception_1: Remove dates
d = datetime.date.today().month
month_now = '{:02d}'.format(d)
d_1 = datetime.date.today().month + 1
d_2 = d_1 + 1
d_3 = d_1 - 2
month_future_1 = '{:02d}'.format(d_1)
month_future_2 = '{:02d}'.format(d_2)
month_past_1 = '{:02d}'.format(d_3)
year = datetime.date.today().year
check_now = str(month_now) + '.' + str(year)
check_future_1 = str(month_future_1) + '.' + str(year)
check_future_2 = str(month_future_2) + '.' + str(year)
check_past_1 = str(month_past_1) + '.' + str(year)

regex_salary = list(map( lambda x: x.replace(check_now, 'False'), regex_salary))
regex_salary = list(map( lambda x: x.replace(check_future_1, 'False'), regex_salary))
regex_salary = list(map( lambda x: x.replace(check_future_2, 'False'), regex_salary))
regex_salary = list(map( lambda x: x.replace(check_past_1, 'False'), regex_salary))

## Exception_2: Remove false numbers containing regex-pattern like: 90610-41 or 7505-40
regex_salary_clean = []
for i in range(len(regex_salary)):
    if (bool(re.search(r'\d{1,9}?(..-\d{1,2})', str(regex_salary[i])))) == True:
        regex_salary_clean.append('False')
    else:
        regex_salary_clean.append(regex_salary[i])
        
        
# Final cleaning of salary: remove punctuation
regex_salary_clean = [y.strip('EUR-€.,<') for y in regex_salary_clean]
regex_salary_clean = [y.strip('brutto') for y in regex_salary_clean]

## Combine lists to singel dataframe
df['salary'] = regex_salary_clean    
df = df[['company', 'job', 'location', 'salary']]

In [167]:
df.head(50)

,company,job,location,salary
0,thyssenkrupp Aufzüge GmbH,Vertragsmanagement (m/w/x),Wien,"2.295,16"
1,legero united,Mitarbeiter/in Arbeitsvorbereitung,Kopfing im Innkreis,"1.561,43"
2,Kotányi GmbH,Maschineneinsteller (w/m),Wolkersdorf im Weinviertel,"2.339,00"
3,KAMPER,Projektleiter Tischlerei (m/w),Wien,"60.000,00"
4,PC Personalmarketing GmbH - Graz,GWH-InstallateurIn,Gleisdorf,2450
5,PC Personalmarketing GmbH - Graz,Softwareentwickler/ in mit Schwerpunkt C / C++...,Gleisdorf,"2400,00"
6,PC Personalmarketing GmbH - Graz,Schaltschrankprüfer/in,Gleisdorf,2500
7,KAMPER,Projektleiter für GU-Projekte (m/w),Wien,"67.000,00"
8,KAMPER,Projektleiter für Haustechnik HKLS (m/w),Wien,"67.000,00"
9,Ordensklinikum Linz GmbH Elisabethinen,Ordinationsassistent/in,Linz,"15.535,10"
